# Computer Exercise 2

In [15]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import matplotlib.pyplot as plt

from numpy import pi, cos, sin, exp, sqrt

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import required files 

In [16]:
KX = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/kx_RW.dat', 
                 sep='\s+', header=None, names=["kx"])

KY = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/ky_RW.dat', 
                 sep='\s+', header=None, names=["ky"])

KZ = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/kz_RW.dat', 
                 sep='\s+', header=None, names=["kz"])

## Let's start plotting!

### Plot the 3D surface map

In [ ]:
SM = plt.